In [3]:
#Dependencies
from bs4 import BeautifulSoup
from requests_html import AsyncHTMLSession


In [4]:
#utilts
import os
import json
def printSoupToHtml(soup):
    #Creating a file if not exist
    if not os.path.exists('soup.html'):
        with open('soup.html', 'w', encoding='utf-8') as file:
            file.write('')
    with open('soup.html', 'w', encoding='utf-8') as file:
        file.write(str(soup))
    return True
def printSoupToJson(soup):
    #Creating a file if not exist
    if not os.path.exists('soup.json'):
        with open('soup.json', 'w', encoding='utf-8') as file:
            file.write('')
    with open('soup.json', 'w', encoding='utf-8') as file:
        file.write(json.dumps(soup))
    return True

In [5]:
#Functions
async def scrapeAmazonSa(url):
    asession = AsyncHTMLSession()
    #attach random user agent and proxy
    asession=AsyncHTMLSession(
        mock_browser=True,
    )
    response = await asession.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    printSoupToHtml(soup)
    #Data
    price = soup.select_one('.aok-align-center span.a-price-whole').get_text().split(".")[0]
    title = soup.select_one('span.product-title-word-break').get_text().strip()
    rating = soup.select_one('#averageCustomerReviews_feature_div span.a-color-base').get_text().strip()
    discription = soup.select_one('ul.a-spacing-mini').get_text().strip()
    print (price, title, rating , discription)
    return  price, title, rating , discription

In [49]:
#Functions
async def scrapeAmazonSa(url , attempt=0):
    try:
        if attempt >= 100:
            # Max retry attempts reached, return empty result or handle accordingly
            return {"images": []}

        asession = AsyncHTMLSession(mock_browser=True)
        response = await asession.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        result = soup.select("#imageBlock img")
        
        for img in result:
            if 'data-a-dynamic-image' in img.attrs:
                dynamic_images = eval(img['data-a-dynamic-image'])
                images=(list(dynamic_images.keys()))
        item_data = {
            "images": images
        }
        return item_data
    
    except Exception as e:
        # Retry recursively with incremented attempt count
        print(f"Error occurred on attempt {attempt + 1}: {e}")
        return await scrapeAmazonSa(url, attempt + 1)       

In [59]:
#Test
import time
url = 'https://www.amazon.sa/%D8%A7%D8%AA%D8%B4-%D8%A8%D9%8A-14-ek1001nx%D8%8C-%D8%AC%D9%8A%D8%AC%D8%A7%D8%A8%D8%A7%D9%8A%D8%AA%D8%8C-%D8%AA%D9%8A%D8%B1%D8%A7%D8%A8%D8%A7%D9%8A%D8%AA%D8%8C/dp/B0CM383CTZ/ref=sr_1_3?refinements=p_n_feature_fifteen_browse-bin%3A16969389031&s=electronics&sr=1-3'
success = False
await scrapeAmazonSa(url)


Error occurred on attempt 1: local variable 'images' referenced before assignment


{'images': ['https://m.media-amazon.com/images/I/71vdMuZUW9L._AC_SX425_.jpg',
  'https://m.media-amazon.com/images/I/71vdMuZUW9L._AC_SX522_.jpg',
  'https://m.media-amazon.com/images/I/71vdMuZUW9L._AC_SX569_.jpg',
  'https://m.media-amazon.com/images/I/71vdMuZUW9L._AC_SX355_.jpg',
  'https://m.media-amazon.com/images/I/71vdMuZUW9L._AC_SX450_.jpg',
  'https://m.media-amazon.com/images/I/71vdMuZUW9L._AC_SX466_.jpg',
  'https://m.media-amazon.com/images/I/71vdMuZUW9L._AC_SX679_.jpg']}